# 数学形态学处理

In [1]:
# -*- conding: utf-8 -*-
import cv2
import numpy as np

img = cv2.imread("triangle.jpg", 0)
ret, img_binary = cv2.threshold(img, 127, 255 ,cv2.THRESH_BINARY_INV) # 变成二值图

print("该图片的尺寸：", img.shape)

该图片的尺寸： (347, 377)


## 膨胀操作：

In [2]:
def inflate(img, struct, x, y):
    """
    膨胀操作， 不影响原图
    img - 操作图像
    struct - 结构元
    x， y - 结构元原点坐标
    return - 膨胀操作后的图片
    """
    img_rows, img_cols = img.shape
    struct_rows, struct_cols = struct.shape
    img_ift = np.zeros((img_rows, img_cols), np.uint8)
    for i in range(img_rows):    # 遍历图像
        for j in range(img_cols): 
            if img.item(i, j) == 255:    # 若该点为前景
#                 print("原图发现前景：", i, j, img.item(i, j))
                for k in range(struct_rows):    # 遍历结构元
                    for l in range(struct_cols):
                        if struct.item(k, l) == 1:    # 找到结构元前景
                            x1 = i + (k - x)    
                            y1 = j + (l - y)
#                             print("应该被改成前景点：", x1, y1)
                            if x1 >= 0 and x1 < img_rows and y1 >= 0 and y1 < img_cols:    # 越界判断
                                img_ift.itemset((x1, y1), 255)    # 直接赋值
    return img_ift

## 腐蚀操作：

In [3]:
def corrode(img, struct, x, y):
    """
    腐蚀操作，不影响原图
    img - 操作图像
    struct - 结构元
    x， y - 结构元原点坐标
    return - 腐蚀操作后的图像
    """
    img_reverse = img.copy()    # 图像拷贝
    cv2.bitwise_not(img_reverse, img_reverse)    # 图像反转
    img_rc = inflate(img_reverse, struct, x, y)    # 对背景进行膨胀操作
    return cv2.bitwise_not(img_rc, img_rc)

## 开运算

In [4]:
def img_open(img, struct, x, y):
    """
    开运算，不影响原图
    img - 操作图像
    struct - 结构元
    x， y - 结构元原点坐标
    return - 开运算后的图像
    """
    img_open = corrode(img, struct, x, y)    # 先腐蚀
    img_open = inflate(img_open, struct, x, y)    # 再膨胀
    return img_open

## 闭运算

In [5]:
def img_close(img, struct, x, y):
    """
    闭运算，不影响原图
    img - 操作图像
    struct - 结构元
    x， y - 结构元原点坐标
    return - 闭运算后的图像
    """
    img_close = inflate(img, struct, x, y)    # 先膨胀
    img_close = corrode(img_close, struct, x, y)    # 再腐蚀
    return img_close

## 测试部分：

结构元

In [6]:
struct = np.array([[1, 1, 1, 1 ,1],
                   [1, 1, 1, 1, 1],
                   [1, 1, 1, 1, 1],
                   [1, 1, 1, 1, 1],
                   [1, 1, 1, 1, 1]])
struct_x, struct_y = 2, 2

测试膨胀操作

In [7]:
img = cv2.imread("triangle.jpg", 0)
ret, img_binary = cv2.threshold(img, 127, 255 ,cv2.THRESH_BINARY_INV)

cv2.imshow("triangle_binary", img_binary)
img_ift = inflate(img_binary, struct, struct_x, struct_y)
cv2.imshow("triangle_inflation", img_ift)

k = cv2.waitKey(0) & 0xFF
if k == 27:
    cv2.destroyAllWindows()

测试腐蚀操作

In [8]:
img = cv2.imread("triangle.jpg", 0)
ret, img_binary = cv2.threshold(img, 127, 255 ,cv2.THRESH_BINARY_INV)

cv2.imshow("triangle_binary", img_binary)
img_cd = corrode(img_binary, struct, struct_x, struct_y)
cv2.imshow("triangle_corrode", img_cd)

k = cv2.waitKey(0) & 0xFF
if k == 27:
    cv2.destroyAllWindows()

测试开运算

In [9]:
img = cv2.imread("triangle.jpg", 0)
ret, img_binary = cv2.threshold(img, 127, 255 ,cv2.THRESH_BINARY_INV)

cv2.imshow("triangle_binary", img_binary)
img_open = img_open(img_binary, struct, struct_x, struct_y)
cv2.imshow("triangle_open", img_open)

k = cv2.waitKey(0) & 0xFF
if k == 27:
    cv2.destroyAllWindows()

测试闭运算

In [10]:
img = cv2.imread("triangle.jpg", 0)
ret, img_binary = cv2.threshold(img, 127, 255 ,cv2.THRESH_BINARY_INV)

cv2.imshow("triangle_binary", img_binary)
img_close = img_close(img_binary, struct, struct_x, struct_y)
cv2.imshow("triangle_close", img_close)

k = cv2.waitKey(0) & 0xFF
if k == 27:
    cv2.destroyAllWindows()